In [1]:
from pulse2percept.models import BiphasicAxonMapModel, AxonMapModel, BiphasicAxonMapSpatial
from pulse2percept.implants import ArgusII
import shapes

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from datetime import datetime
import math

In [3]:
data = shapes.load_shapes("../data/shapes.h5", implant='ArgusII', stim_class=['MultiElectrode', 'SpatialSummation'])
data.head()

,subject,amp1,amp2,area,compactness,date,eccentricity,elec_delay,electrode1,electrode2,...,image,implant,num_regions,orientation,pdur,stim_class,trial,x_center,y_center,img_shape
0,12-005,1.5,1.5,434,0.783044,20091028,0.700223,0.0,A10,C10,...,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",ArgusII,1,-0.150516,0.45,MultiElectrode,1,256.0,192.0,"(384, 512)"
1,12-005,1.5,1.5,595,0.536552,20091028,0.713548,0.0,A10,C10,...,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",ArgusII,1,-0.760744,0.45,MultiElectrode,15,256.0,192.0,"(384, 512)"
2,12-005,1.5,1.5,459,0.465158,20091028,0.753356,0.0,A10,C10,...,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",ArgusII,1,0.341745,0.45,MultiElectrode,33,256.0,192.0,"(384, 512)"
3,12-005,1.5,1.5,114,0.311651,20091028,0.927572,0.0,A10,C10,...,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",ArgusII,1,0.138759,0.45,MultiElectrode,37,256.0,192.0,"(384, 512)"
4,12-005,1.5,1.5,435,0.407359,20091028,0.833744,0.0,A10,C10,...,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",ArgusII,1,-0.589053,0.45,MultiElectrode,60,256.0,192.0,"(384, 512)"


In [6]:
# load all the data up
for subject in data['subject'].unique():
    implant, model = shapes.model_from_params(shapes.subject_params[subject], biphasic=False)
    locs = data['subject'] == subject
    subj_data = data.loc[locs]
    data.loc[locs,['d_retina', 'd_perp_avg', 'd_tan_avg']] = shapes.dist_perpendicular_tangential(subj_data['electrode1'], subj_data['electrode2'], implant, model, strategy='average')
    data.loc[locs,['d_retina', 'd_perp', 'd_tan']] = shapes.dist_perpendicular_tangential(subj_data['electrode1'], subj_data['electrode2'], implant, model, strategy='upstream')
    data.loc[locs,['d_retina', 'd_across_axonal', 'd_along_axonal']] = shapes.dist_across_along(subj_data['electrode1'], subj_data['electrode2'], implant, model, strategy='axonal')
    data.loc[locs,['d_retina', 'd_across_radial', 'd_along_radial']] = shapes.dist_across_along(subj_data['electrode1'], subj_data['electrode2'], implant, model, strategy='radial')
    data.loc[locs,['d_retina', 'd_across_mixed', 'd_along_mixed']] = shapes.dist_across_along(subj_data['electrode1'], subj_data['electrode2'], implant, model, strategy='mixed')


(-234.42254813070167, -2546.238726160018)
[ -254.94 -3451.59]
[-1032.14 -3372.48]
(-234.42254813070167, -2546.238726160018)
[ -254.94 -3451.59]
[-1032.14 -3372.48]
(-234.42254813070167, -2546.238726160018)
[ -254.94 -3451.59]
[-1032.14 -3372.48]
(-234.42254813070167, -2546.238726160018)
[ -254.94 -3451.59]
[-1032.14 -3372.48]
(-234.42254813070167, -2546.238726160018)
[ -254.94 -3451.59]
[-1032.14 -3372.48]
(-234.42254813070167, -2546.238726160018)
[ -126.62 -2037.71]
[-1048.6  -1755.48]
(-234.42254813070167, -2546.238726160018)
[ -126.62 -2037.71]
[-1048.6  -1755.48]
(-234.42254813070167, -2546.238726160018)
[ -126.62 -2037.71]
[-1048.6  -1755.48]
(-234.42254813070167, -2546.238726160018)
[ -126.62 -2037.71]
[-1048.6  -1755.48]
(-234.42254813070167, -2546.238726160018)
[ -126.62 -2037.71]
[-1048.6  -1755.48]
(-276.99801467310476, -107.09188153407399)
[ -742.44 -1011.63]
[-1496.65  -534.4 ]
(-276.99801467310476, -107.09188153407399)
[ -742.44 -1011.63]
[-1496.65  -534.4 ]
(-276.99801467

AssertionError: 